<a href="https://colab.research.google.com/github/acoiman/swb/blob/main/surface_water_balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estimación el Balance de Agua Superficial y el Volumen de Agua de las Cuencas en Sur América (WWF HydroSHEDS Nivel 2).

A través de esta notebook vamos a hacer una aproximación al cálculo  del Balance de Agua Superficial (BAS) y el Volumen de Agua (VA) en las cuencas de sur américa correspondientes al nivel 2 de conjunto de los datos WWF HydroSHEDS. Este análisis lo vamos a efectuar en una serie temporal mensual. 

Para lograr este objetivo vamos a usar la API Python de Google Earh Engine (GEE) y otras librerías Python. El conjunto de datos que vamos a emplear es el siguiente:

- Cuencas de la colección [WWF HydroSHEDS](https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_v1_Basins_hybas_2). Este conjunto de datos proporciona polígonos de cuencas hidrográficas jerárquicas y anidadas, basados en datos ráster de resolución de 15 segundos de arco (aproximadamente 500 m en el ecuador). Las cuencas hidrográficas van desde el nivel 1 (grueso) al nivel 12 (detallado), utilizando el código Pfastetter.
- [GLDAS-2.1: Global Land Data Assimilation System](https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H). De los datos  GLDAS-2.1  vamos a selecionar la proporción total de precipitación (`Rainf_f_tavg`), la evapotranspiración (`Evap_tavg`) y la escorrentía superficial de la lluvia (`Qs_acc`). La resolución espacial de estos datos es de 0.25 grados de arco, aproximadamente 30 Km.

El cálculo del BAS es el siguiente:

$${BAS(mm/mes)} = {{precipitación} - {evapotranspiración} - {escorrentía}}*.$$

<br/>

BAS es una medida de flujo de agua expresado en ${kg/m^2}$ que es equivalente a ${mm}$, cuando lo calculamos por mes obtenemos ${mm/mes}$.

<br/>

El cálculo del VA es el siguiente: 

$${VA(m^3/mes)} = {{BAS(mm/mes)} / 1000 * {AreaCuenca(m^2)}}.$$

<br/>

Lo anterior es  una primera aproximación al cálculo del volumen de agua en la cuenca porque no se toman en cuenta otros factores como el suelo y las aguas subterráneas.

Estos datos se cruzarán con otros datos de puntos de calor y área quemadas por cuenca con el fin de establecer patrones entre los incendios forestales y la disponibilidad de agua.

<br/>

<font size="2">* Fuente: NASA's Applied Remote Sensing Training Program (ARSET)</font> 

Para iniciar procedamos a instalar algunos paquetes necesarios.

In [ ]:
# installl packages
!pip install geopandas
!pip install eeconvert
!pip install geemap
!pip install arrow

In [ ]:
!pip uninstall tornado
!pip install tornado==4.5.3

Accedemos a los servicios de GEE.

In [2]:
# Authenticate to Earth Engine
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=vuAc0w4SNubpJ9mjWwn3oXSVegjFLEesNy9pRtT7i-Q&tc=PXuFZIeUFttDXW6QoQWlh_73FTr_aT2b-gxyA8pKxaY&cc=5SreymJquiiiRi7RVbryAJ_u09grt3rroymW7kJJCW0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJoRQJ-cGwbPqEv44nL_VdCh1-qgZWatlVGuK5GKBqpxzbA0v7Wmug8

Successfully saved authorization token.


Procedamos a importar algunos paquetes necesarios.

In [3]:
# import packages
import matplotlib.pyplot as plt
from pprint import  pprint
import eeconvert
import ee
import folium
import geemap.geemap as geemap
import geopandas
from datetime import date
from datetime import datetime
import arrow
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

Inicialicemos GEE. 

In [4]:
# initialize GEE
ee.Initialize()

Accedamos a los servicios de almacenamiento de Google Drive.

In [5]:
# Authenticate to Google Drive
# Mount Google Drive
from google.colab import drive # import drive from google colab

# default location for the drive
ROOT = "/content/drive" 
print(ROOT) # print content of ROOT (optional)

drive.mount(ROOT) # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


Cambiémonos a nuestra carpeta de trabajo. Se debe crear una carpeta con el nombre Colab_Notebooks y dentro de esta una carpeta con el nombre Taller_GEE_Inc_2021, en caso contrario configure su ruta particular.

In [6]:
%cd "drive/MyDrive/Colab_Notebooks/visita_ig"

/content/drive/MyDrive/Colab_Notebooks/visita_ig


In [7]:
level = ['Nivel 2', 'Nivel 3', 'Nivel 4', 'Nivel 5', 'Nivel 6','Nivel 7', 'Nivel 8', 'Nivel 9', 'Nivel 10', 'Nivel 11', 'Nivel 12']

In [8]:
# set up a Jupyter Notebook dropdown widget to pick up a basin level
w = widgets.Dropdown(
    options= level,
    value='Nivel 2'

)
print('Seleccione un Nivel de Cuencas')
display(w)

Seleccione un Nivel de Cuencas


Dropdown(options=('Nivel 2', 'Nivel 3', 'Nivel 4', 'Nivel 5', 'Nivel 6', 'Nivel 7', 'Nivel 8', 'Nivel 9', 'Niv…

In [9]:
if w.value == level[0]:
  endurl = 'hybas_2'
elif w.value == level[1]:
  endurl = 'hybas_3'
elif w.value == level[2]:
  endurl = 'hybas_4'
elif w.value == level[3]:
  endurl = 'hybas_5'
elif w.value == level[4]:
  endurl = 'hybas_6'
elif w.value == level[5]:
  endurl = 'hybas_7'
elif w.value == level[6]:
  endurl = 'hybas_8'
elif w.value == level[7]:
  endurl = 'hybas_9'
elif w.value == level[8]:
  endurl = 'hybas_10'
elif w.value == level[9]:
  endurl = 'hybas_11'
else:
  endurl = 'hybas_12'

In [10]:
colSnippet = 'WWF/HydroSHEDS/v1/Basins/' + endurl
print(colSnippet)

WWF/HydroSHEDS/v1/Basins/hybas_2


Creamos una lista con los IDs de las cuencas en Suramérica, filtramos la colección `WWF HydroSHEDS Basins del nivel 2` de acuerdo a la lista y luego unimos para obtener el polígono de Sur América que será empleado para cortar los datos del GLDAS.

In [11]:
basin_id = [6020029280, 6020017370, 6020021870, 6020008320,  6020014330, 6020000010, 6020006540];

southAmer = (ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_2')
         .filter(ee.Filter.inList('HYBAS_ID',basin_id))
         .union()
)

Creamos un polígono para filtrar las cuencas de nivel seleccionado.

In [12]:
geometry = ee.Geometry.Polygon([[[-73.29873105727071, 13.479144572417201],
                                [-75.44126165031963, 11.486147543732507],
                                [-75.43833095188438, 7.191350227504306],
                                [-78.57216855727071, 4.284821423021577],
                                [-85.25185605727071, -5.371263035360049],
                                [-76.28701230727071, -24.30326838962132],
                                [-77.69326230727071, -56.032905509323086],
                                [-63.279199807270714, -59.04972038791961],
                                [-55.544824807270714, -47.94300457794506],
                                [-54.314356057270714, -38.35563126516214],
                                [-26.540918557270714, -5.136947070874553],
                                [-49.392481057270714, 6.9678245412504305],
                                [-65.03701230727071, 14.219615877891318]]])

In [42]:
# create map to visualize the polygon (geometry)
Map = geemap.Map(center=(-9.84, -62.45), zoom=3)

# add layers to map
Map.addLayer(geometry, {}, "Polígono")

# visualize map
Map

Map(center=[-9.84, -62.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [13]:
# filter FeatureCollection using the polygon
basins = (ee.FeatureCollection(colSnippet)
          .filterBounds(geometry)
)

Ahora visualicemos el polígono de Suramérica y las cuencas de Suramérica en un mapa.

In [56]:
# create map
Map = geemap.Map(center=(-9.84, -62.45), zoom=3)

# layer styles 
style = {'color': 'black', 'fillColor': "00000000"} # basins
vis_sa = {
    'color': '000000', 
    'colorOpacity': 1,
    'lineType': 'solid', 
    'fillColorOpacity': 1,
    'width': 1    
} # South America

# add layers to map
Map.addLayer(basins.style(**style), {}, "Cuencas") # basins
Map.addLayer(southAmer , vis_sa, 'Sur América ')

# add labels and label style
Map.add_labels(
    basins,
    "HYBAS_ID",
    font_size="12pt",
    font_color="blue",
    font_family="arial",
    font_weight="bold",
)

# add map controls
Map.addLayerControl()

# visualize map
Map

Map(center=[-9.84, -62.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

Seleccionamos la fechas de nuestro periodo de estudio. Si el periodo de tiempo es entre el 01 de enero y el 31 enero, la fecha final debe ser un día adicional de la fecha final deseada, en este caso 01 de febrero.

In [15]:
start = widgets.DatePicker(
    description='Fecha Inicial',
    disabled=False
)

In [16]:
display(start)

DatePicker(value=None, description='Fecha Inicial')

In [17]:
print(start.value)

2022-09-01


In [18]:
end = widgets.DatePicker(
    description='Fecha Final',
    disabled=False
)

In [19]:
display(end)

DatePicker(value=None, description='Fecha Final')

In [20]:
print(end.value)

2022-09-30


Calculemos las fechas del primer día del mes entre un periodo de tiempo determinado.

In [21]:
def ini_dates(start, end):
    
    '''function to create a pandas series containing the first day of a month for given period
    Argumets:
    start: initial date in format 'year-month-day'
    end: final date in format 'year-month-day'
    
    return: a pandas Series with dates in format 'year-month-day'
        
    Example: if you want to get the first day between January and February you should enter 
    '2000-01-01' as initial date and '2000-03-01' as final date
    '''
    range= pd.date_range(start,end , freq='1M')-pd.offsets.MonthBegin(1)
    serie = pd.Series(range.format())
    
    return serie

In [22]:
startDates = ini_dates(start.value,end.value)

In [23]:
startDates

0    2022-09-01
dtype: object

Calculemos las fechas del último día del mes entre un periodo de tiempo determinado.

In [24]:
def fin_dates(start, end):
    
    '''function to create a pandas series containing the last day of a month for given period
    Argumets:
    start: initial date in format 'year-month-day'
    end: final date in format 'year-month-day'
    
    return: a pandas Series with dates in format 'year-month-day'
        
    Example: if you want to get the last day between January and February you should enter 
    '2000-01-01' as initial date and '2000-03-01' as final date
    '''
    range= pd.date_range(start,end , freq='1M')-pd.offsets.MonthEnd(0)
    serie = pd.Series(range.format())
    
    return serie

In [25]:
endDates = fin_dates(start.value,end.value)

In [26]:
endDates

0    2022-09-30
dtype: object

En esta sección vamos a calcular Balance de Agua Superficial y el Volumen de Agua por mes para el período y área de estudio. Luego calcularemos las estadísticas zonales (media) del BAS para cada cuenca. Convertiremos el FeatureCollection a un GeoDataFrame para calcular el área por cuenca y el volumen de agua por cuenca. Luego se almacenará toda esta serie temporal en una lista.

In [27]:
# list to store dataframe values
lst = []
 
for d1, d2 in zip(startDates, endDates):
    # d1: start date
    # d2: end date

    # calculate number of days between d1 and d2
    a = arrow.get(d1)
    b = arrow.get(d2)
    delta = (b-a)
    numdays = delta.days

    # GLDAS-2.1: Global Land Data Assimilation System
    # https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H
    dataset = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filter(ee.Filter.date(d1, d2)) 
    # select total precipitation rate and calculate the mean
    precip_rate = dataset.select('Rainf_f_tavg').mean()
    # clip to all basins
    precip_clip = precip_rate.clip(southAmer)
    # convert units to get monthly data
    # precipitation is in kg m^2 s^-1  accumulated over 3 hour interval 
    # Rainf_f_tavg (month){kg/m2} =
    # Rainf_f_tavg (month){kg/m2/sec} * 10800{sec/3hr} * 8{3hr/day} * numdays{days}
    precip = precip_clip.multiply(10800).multiply(8).multiply(numdays)
   
    # select evapotranspiration and calculate the mean
    evap = dataset.select('Evap_tavg').mean()
    # clip to all basins
    evap_clip =  evap.clip(southAmer)  
    # convert units to get monthly data
    # evapotraspiration is in kg m^2 s^-1  accumulated over 3 hour interval 
    # Evap_tavg (month){kg/m2} =
    # Evap_tavg (month){kg/m2/sec} * 10800{sec/3hr} * 8{3hr/day} * numdays{days}
    ET =  evap_clip.multiply(10800).multiply(8).multiply(numdays) 
   
    # select storm surface runoff and calculate the mean
    ssr = dataset.select('Qs_acc').mean()
    # clip to all basins
    ssr_clip  =  ssr.clip(southAmer)
    # convert units to get monthly data
    # runoff is in kg m^2  accumulated over 3 hour interval 
    # Qs_acc (month){kg/m2} = Qs_acc (month){kg/m2/3hr} * 8{3hr/day} * numdays{days}
    runoff =  ssr_clip.multiply(8).multiply(numdays)
   
    # calculate Surface Water Balance
    SWB= precip.subtract(ET);
    WB = SWB.subtract(runoff).rename('WB');
   
    # add reducer output to the features in the collection.
    # add the mean of  Surface Water Balance to each basin
    saBasinsWB = WB.reduceRegions(**{
        'collection': basins,
        'reducer': ee.Reducer.mean(),
        'scale': 30000
    })
    # conver fc to gdf
    df = eeconvert.fcToGdf(saBasinsWB)

    # create FECHA column
    df['FECHA'] =  d1.replace('-', '')[:-2]
   
    # reproject to South America Albers Equal Area Conic
    df_proj= df.to_crs("+proj=aea +lat_1=-5 +lat_2=-42 +lat_0=-32 +lon_0=-60 +x_0=0 +y_0=0 +ellps=aust_SA +units=m +no_defs")
   
    # calculate area in m^2
    df_proj["AREA_M2"] = df_proj['geometry'].area

    # calculate water volume in m^3
    df_proj["VOLUMEN_AGUA_M3"] =  (df_proj['mean'] / 1000) * df_proj['AREA_M2']
   
    # select and rename columns
    df_proj = df_proj[['HYBAS_ID', 'mean', 'FECHA', 'VOLUMEN_AGUA_M3']]
    df_proj = df_proj.rename(columns={'mean': 'WB'})

    # reorder columns
    df_final = df_proj[['FECHA', 'HYBAS_ID','WB', 'VOLUMEN_AGUA_M3']]
   
    # df values to list
    dflist = df_final.values.tolist()
    # append dflist to another list
    lst.append(dflist)

Eliminemos los paréntesis adicionales de la lista.

In [28]:
# flatten list
final_list = [i for sublist in lst for i in sublist]

A partir de la lista anterior vamos a crear una DataFrame, luego los visualizaremos. 

In [29]:
# create  new dataframe from final_list
df2 = pd.DataFrame(final_list, columns =['FECHA', 'HYBAS_ID','WB', 'VOLUMEN_AGUA_M3'])

In [30]:
# visualize new dataframe
df2.head()

,FECHA,HYBAS_ID,WB,VOLUMEN_AGUA_M3
0,202209,6020000010,35.903540,7.565374e+10
1,202209,6020006540,-30.027601,-2.067651e+11
2,202209,6020008320,-5.803652,-1.217733e+10
3,202209,6020014330,10.879021,3.919900e+10
4,202209,6020017370,-4.494863,-6.803186e+09


Para una mejor compresión, a continuación, crearemos un campo con el VA en millardo de metros cúbicos. Un millardo es el número natural equivalente a mil millones (1 000 000 000) o, en notación científica, $10^{9}$.

In [31]:
df2['VOLUMEN_AGUA_MM3'] = round(df2['VOLUMEN_AGUA_M3'] / 1000000000, 2)
df2.head()

,FECHA,HYBAS_ID,WB,VOLUMEN_AGUA_M3,VOLUMEN_AGUA_MM3
0,202209,6020000010,35.903540,7.565374e+10,75.65
1,202209,6020006540,-30.027601,-2.067651e+11,-206.77
2,202209,6020008320,-5.803652,-1.217733e+10,-12.18
3,202209,6020014330,10.879021,3.919900e+10,39.20
4,202209,6020017370,-4.494863,-6.803186e+09,-6.80


A continuación, procedemos a calcular los [cuartiles](https://support.minitab.com/es-mx/minitab/18/help-and-how-to/graphs/how-to/boxplot/interpret-the-results/quartiles/) para  a clasificar el volumen de agua en millardos de ${m^3}$ de toda la serie temporal.


In [33]:
# calculate quantiles
dfquantiles = df2['VOLUMEN_AGUA_MM3'].quantile([0.25, 0.50, 0.75]) 

# adjust quantile 0.25 to 0 to include all negative data into a category
dfquantiles[0.25] =0
dfquantiles

0.25     0.000
0.50    19.630
0.75    56.395
Name: VOLUMEN_AGUA_MM3, dtype: float64

Con estos resultado podemos constuir la siguiente tabla de clasificación. Es importante indicar que la clasificación de los cuartiles va a depender del rango de los datos.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-g0ma{border-color:inherit;font-size:xx-large;font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-8afn{border-color:inherit;font-size:xx-large;text-align:left;vertical-align:top}
@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style>
<div class="tg-wrap"><table class="tg">
<thead>
  <tr>
    <th class="tg-g0ma">Rango</th>
    <th class="tg-g0ma">Clase</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-8afn">&lt; 0</td>
    <td class="tg-8afn">Baja</td>
  </tr>
  <tr>
    <td class="tg-8afn">0 - 31.73</td>
    <td class="tg-8afn">Media</td>
  </tr>
  <tr>
    <td class="tg-8afn">31.73 - 113.16</td>
    <td class="tg-8afn">Alto</td>
  </tr>
  <tr>
    <td class="tg-8afn">&gt; 113.16</td>
    <td class="tg-8afn">Muy Alto</td>
  </tr>
</tbody>
</table></div>

In [34]:
#labeling of groups
df2['CLASE_VAMM3'] = 'Muy Alto'
df2['CLASE_VAMM3'][df2['VOLUMEN_AGUA_MM3'] < dfquantiles[0.75]] = 'Alto'   
df2['CLASE_VAMM3'][df2['VOLUMEN_AGUA_MM3'] < dfquantiles[0.50]] = 'Medio'   
df2['CLASE_VAMM3'][df2['VOLUMEN_AGUA_MM3'] < dfquantiles[0.25]] = 'Bajo' 
df2.head()

,FECHA,HYBAS_ID,WB,VOLUMEN_AGUA_M3,VOLUMEN_AGUA_MM3,CLASE_VAMM3
0,202209,6020000010,35.903540,7.565374e+10,75.65,Muy Alto
1,202209,6020006540,-30.027601,-2.067651e+11,-206.77,Bajo
2,202209,6020008320,-5.803652,-1.217733e+10,-12.18,Bajo
3,202209,6020014330,10.879021,3.919900e+10,39.20,Alto
4,202209,6020017370,-4.494863,-6.803186e+09,-6.80,Bajo


Finalmente exportemos los datos en formato csv

In [ ]:
df2.to_csv('wb2000_2019.csv', index=False)